In [1]:
import duckdb

duckdb.__version__

'v0.7.1-dev648'

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [3]:
%sql duckdb:///snb.duckdb

%sql load 'sqlpgq'

""


In [4]:
%%sql
    CREATE PROPERTY GRAPH social_network
    VERTEX TABLES (
        Person PROPERTIES (id, firstName, lastName) LABEL Person,

        University PROPERTIES (id, name) LABEL University,

        Tag Properties (id, name, url) LABEL Tag
    )
    EDGE TABLES (
       Person_studyAt_University 
            SOURCE KEY (PersonId) REFERENCES Person (id)
            DESTINATION KEY (UniversityId) REFERENCES University (id)
            PROPERTIES (classYear)
            LABEL Person_studyAt_University,

        Person_hasInterest_Tag 
            SOURCE KEY (PersonId) REFERENCES Person (id)
            DESTINATION KEY (TagId) REFERENCES Tag (id) 
            PROPERTIES (creationDate)
            LABEL Person_hasInterest_Tag,

        Person_knows_Person 
            SOURCE KEY (Person1id) REFERENCES Person (id)
            DESTINATION KEY (Person2id) REFERENCES Person (id)
            LABEL Person_knows_Person
    )

""


In [9]:
%%sql
    SELECT pg.bob, pg.bob_id, count(pg.friend_name) as number_of_friends

    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person WHERE p.firstName = 'Bob')-[k:Person_knows_Person]->(p2:person)
    COLUMNS (p.firstName as bob, p.id as bob_id, p2.firstName as friend_name) 
    ) pg

    GROUP BY pg.bob, pg.bob_id

,bob,bob_id,number_of_friends
0,Bob,977,100
1,Bob,10995116285001,10
2,Bob,28587302324999,1


In [24]:
%%sql
    query_plan << EXPLAIN SELECT pg.bob, pg.bob_id, count(pg.friend_name) as number_of_friends

    FROM GRAPH_TABLE (social_network,
    MATCH (p:Person WHERE p.firstName = 'Bob')-[k:Person_knows_Person]->(p2:person)
    COLUMNS (p.firstName as bob, p.id as bob_id, p2.firstName as friend_name) 
    ) pg

    GROUP BY pg.bob, pg.bob_id

In [25]:
print(query_plan['explain_value'][0])

┌───────────────────────────┐                                                          
│       HASH_GROUP_BY       │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│             #0            │                                                          
│             #1            │                                                          
│        count_star()       │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│            bob            │                                                          
│  

In [30]:
%%sql SELECT pg.firstname, pg.id, count(pg.reachable_people)
FROM GRAPH_TABLE (social_network,
MATCH (p:Person)-[k:Person_knows_Person]->{1,4}(p2:person)
WHERE p.firstname = 'Bob'
COLUMNS (p.firstName as firstname, p.id as id, p2.firstName as reachable_people) 
) pg
GROUP BY ALL;

,firstname,id,count(pg.reachable_people)
0,Bob,977,8291
1,Bob,10995116285001,5079
2,Bob,28587302324999,87
